## A* agent

In [5]:
import os
import re
import subprocess
from csv import writer

import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

from statistics import median
from scipy.stats import mode

In [18]:
# Parameters for generator
start_range = '0'
end_range = '149'
no_levels = str(int(end_range) - int(start_range) + 1) 
reps_per_level = '1'

using_generator = 'false'
agent_timer = '20'

process = subprocess.Popen(['/Library/Java/JavaVirtualMachines/jdk-11.0.2.jdk/Contents/Home/bin/java', '-cp', 
                            '/Users/uni/Library/Application Support/Code/User/workspaceStorage/a3db8958a2adb6a271882e80d52f6316/redhat.java/jdt_ws/ECS7002P-MarioAI-master_b5c98599/bin', 
                            'RunLevels', start_range, end_range, no_levels, reps_per_level, using_generator, agent_timer], stdout=subprocess.PIPE)

output, _ = process.communicate()

with open('output.txt', 'w') as f:
    f.write(output.decode())

## Metrics

### Leniency

Leniency is the overall difficulty level for the player. Leniency will be calculated by quantifying the places where the player could die:
- Number of enemies
- Number of gaps
- Width of gaps

Normally, the type of enemy would also matter but in the level representation the type of enemies has been generalized away. 

In [7]:
def get_gap_lengths(string):
    gap_lengths = []
    current_gap_length = 0
    for char in string:
        if char == '-':
            current_gap_length += 1
        else:
            if current_gap_length > 0:
                gap_lengths.append(current_gap_length)
                current_gap_length = 0
    return gap_lengths

In [8]:
def leniency(level):
    level_length = len(level[0])
    # Number of enemies
    enemy = 'E'
    cannon = 'B'
    enemy_count = 0

    for y in level:
        for x in y:
            if x == enemy:
                enemy_count += 1
            elif x == cannon:
                enemy_count += 1.8
    
    # Width of gaps
    ground = level[-1]
    gap_lengths = get_gap_lengths(ground)

    # Number of gaps
    gap_count = len(gap_lengths)

    if gap_count == 0:
        average_gap = 0
    else:
        average_gap = sum(gap_lengths) / gap_count

    # print((0.6 * (enemy_count / 10)), (0.4 * (average_gap / 10)), (0.5 * (level_length / 90)))
    weighted_sum = (0.5 * (enemy_count / 10)) + (0.5 * (average_gap / 10)) + (0.5 * (level_length / 90))

    return weighted_sum

## Linearity

Linearity tells how much verticality the map has. A very linear level is relatively flat, and a non-linear level has a lot of vertical platforms. We will calculate the linearity by fitting a line through the end-points of each platform and calculating the R2 goodness of fit measure. A low linearity value would mean that there are many irregularities. A high linearity value would mean the level is relatively flat. 

In [9]:
def find_platforms(level):
    platforms = []
    height = len(level)

    for layer in level:
        height -= 1 
        platform_length = 0

        if height == 0: 
            break

        for index, char in enumerate(layer):
            if char == 'S' or char == 'Q' or char == '?' or char == 'X':
                platform_length += 1
            else:
                if platform_length > 0 and platform_length <= 1:
                    platform_length = 0

                elif platform_length >= 2:
                    platforms.append((index-1, height))
                    platform_length = 0

    return platforms

In [10]:
def linearity(level):
    r2_scores = []

    platforms = find_platforms(level)
    
    if len(platforms) < 2:
        return 0.95
    
    if not platforms:
        return 1.0

    Xs = []
    ys = []
    
    for platform in platforms:
        X, y = platform
        Xs.append(X)
        ys.append(y)
    
    X = np.array(Xs)
    y = np.array(ys)

    X = X.reshape(-1, 1)
        
    model = LinearRegression().fit(X, y)
    predicted_end_y = model.predict(X)
    
    r2 = r2_score(y, predicted_end_y)
    r2_scores.append(r2)
    
    linearity_metric = sum(r2_scores) / len(r2_scores)
    
    return linearity_metric

## Density

Density is defined by the number of hills of the level

In [11]:
def density(level):
    platforms = []
    height = len(level)
    length = len(level[0])

    for layer in level:
        height -= 1 
        platform_length = 0

        if height == 0: 
            break

        for index, char in enumerate(layer):
            if char == 'X':
                platform_length += 1
            else:
                if platform_length > 0 and platform_length <= 1:
                    platform_length = 0

                elif platform_length >= 2:
                    platforms.append((index-1, height))
                    platform_length = 0

    return len(platforms)/length

## Evaluate each level

In [12]:
directory = r"/Users/uni/Documents/Thesis/self/generated_files"

summary = {}
 
for name in os.listdir(directory):
    with open(os.path.join(directory, name)) as f:
        level = f.read()
        processed_level = level.split('\n')

        leni = leniency(processed_level)
        line = linearity(processed_level)
        dens = density(processed_level)

        summary[name] = (leni, line, dens)

        # print(f"Statistics of '{name}'")
        # print(f'Leniency: {leni}\nLinearity: {line}\nDensity: {dens}\n')

UnicodeDecodeError: 'utf-8' codec can't decode byte 0x80 in position 3131: invalid start byte

Exception in thread "main" java.lang.NullPointerException
	at engine.core.MarioLevel.clone(MarioLevel.java:287)
	at engine.core.MarioWorld.clone(MarioWorld.java:116)
	at engine.core.MarioGame.gameLoop(MarioGame.java:249)
	at engine.core.MarioGame.runGame(MarioGame.java:222)
	at engine.core.MarioGame.runGame(MarioGame.java:180)
	at RunLevels.main(RunLevels.java:82)


In [ ]:
nan_values = []

for key, values in summary.items():
    for value in values:
        if np.isnan(value):
            nan_values.append((key, values))

print("NaN values in the original dictionary:")
for nan_value in nan_values:
    print(nan_value)

NaN values in the original dictionary:


In [ ]:
# Extracting leniency, linearity, and density values into separate lists
leniency_values = [value[0] for value in summary.values()]
linearity_values = [value[1] for value in summary.values()]
density_values = [value[2] for value in summary.values()]

# Calculating average
average_leniency = np.mean(leniency_values)
average_linearity = np.mean(linearity_values)
average_density = np.mean(density_values)

# Calculating median
median_leniency = median(leniency_values)
median_linearity = median(linearity_values)
median_density = median(density_values)

# Calculating mode
mode_leniency = mode(leniency_values)
mode_linearity = mode(linearity_values)
mode_density = mode(density_values)

print("Average of leniency values:", average_leniency)
print("Average of linearity values:", average_linearity)
print("Average of density values:", average_density)
print()
print("Median of leniency values:", median_leniency)
print("Median of linearity values:", median_linearity)
print("Median of density values:", median_density)
print()
print("Mode of leniency values:", mode_leniency.mode, f'Count: {mode_leniency.count}')
print("Mode of linearity values:", mode_linearity.mode, f'Count: {mode_linearity.count}')
print("Mode of density values:", mode_density.mode, f'Count: {mode_density.count}')


Average of leniency values: 1.7302947811447813
Average of linearity values: 0.4730643187643735
Average of density values: 0.03617981721581312

Median of leniency values: 1.3557638888888888
Median of linearity values: 0.36360709874119557
Median of density values: 0.03187301587301587

Mode of leniency values: 0.6722222222222223 Count: 3
Mode of linearity values: 0.95 Count: 31
Mode of density values: 0.0 Count: 41


## Saving data in CSV

In [ ]:
with open('output.txt', 'r') as file:
    data = file.read()

# game_result_re = re.compile(r'(\d+)/50;1/1: (\w+|TIME_OUT)')
summary_re = re.compile(r'(\w+(?: \w+)*): ([\d.]+)')

# game_results = game_result_re.findall(data)
# print("Game Results:")
# for result in game_results:
#     print(result)

summary_stats = summary_re.findall(data)
print("\nSummary Statistics:")
for stat in summary_stats:
    print(stat)


Summary Statistics:
('Win Rate', '0.19333333')
('Percentage Completion', '60.10412597656249')
('Lives', '0.0')
('Coins', '0.37333333333333335')
('Remaining Time', '16.346666666666668')
('Mario State', '0.0')
('Mushrooms', '0.0')
('Fire Flowers', '0.0')
('Total Kills', '0.12')
('Stomp Kills', '0.12')
('Fireball Kills', '0.0')
('Shell Kills', '0.0')
('Fall Kills', '0.0')
('Bricks', '0.0')
('Jumps', '5.073333333333333')
('Max X Jump', '111.89453125')
('Max Air Time', '10.5')
('Brick bump', '0.0')
('Question block bump', '0.0')
('Mario hits', '0.013333333333333334')


In [ ]:
snaking = True
path_info = True
column_nr = True

List = [snaking, path_info, column_nr,
        average_leniency, average_linearity, average_density, 
        median_leniency, median_linearity, median_density, 
        (mode_leniency.mode, mode_leniency.count,), (mode_linearity.mode, mode_linearity.count), (mode_density.mode, mode_density.count),
        summary_stats[0][1], summary_stats[1][1]]

with open('experiment_data.csv', 'a') as f_object:
    writer_object = writer(f_object)
    writer_object.writerow(List)
    f_object.close()

In [ ]:
pd.read_csv('experiment_data.csv')

,snaking,path_info,column_nr,avg_leniency,avg_linearity,avg_density,med_leniency,med_linearity,med_density,mod_leniency,mod_linearity,mod_density,win_rate,completion_percentage
0,False,False,False,2.769516,0.795002,0.014304,2.682639,0.950000,0.000000,"(1.6583333333333332, 2)","(0.95, 75)","(0.0, 88)",0.520000,68.151993
1,False,False,True,2.126233,0.606855,0.017368,2.109722,0.950000,0.005376,"(1.6166666666666667, 2)","(1.0, 40)","(0.0, 73)",0.586667,82.676524
2,False,True,False,1.490710,0.578322,0.031037,1.144444,0.834751,0.015811,"(1.0555555555555556, 3)","(0.95, 50)","(0.0, 57)",0.466667,73.048925
3,False,True,True,1.572619,0.652361,0.024681,1.337500,0.950000,0.015446,"(1.238888888888889, 4)","(0.95, 41)","(0.0, 58)",0.413333,59.358449
4,True,False,False,2.774229,0.247440,0.072291,2.506667,0.156778,0.074167,"(2.5944444444444446, 2)","(1.0, 7)","(0.0, 19)",0.106667,45.949361
5,True,False,True,2.902470,0.386461,0.032295,2.438056,0.232388,0.018987,"(1.1111111111111112, 1)","(1.0, 22)","(0.0, 48)",0.160000,40.790878
6,True,True,False,1.397203,0.682603,0.016139,1.238194,0.950000,0.000000,"(0.861111111111111, 2)","(0.95, 70)","(0.0, 77)",0.240000,72.913081
7,True,True,True,1.730295,0.473064,0.036180,1.355764,0.363607,0.031873,"(0.6722222222222223, 3)","(0.95, 31)","(0.0, 41)",0.193333,60.104126
